In [19]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics
from calitp_data_analysis import calitp_color_palette

# from rt_analysis import rt_filter_map_plot
import update_vars_index

from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

import altair as alt
from calitp_data_analysis.tables import tbls

In [20]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [21]:
## parameters cell
itp_id = 182

In [22]:
analysis_date = update_vars_index.ANALYSIS_DATE
import datetime as dt

In [23]:
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet')

In [24]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet')

In [27]:
speedmap_index >> filter(_.name.str.contains('LA Metro'))

,organization_name,name,base64_url,caltrans_district,status,analysis_date
1,Los Angeles County Metropolitan Transportation...,LA Metro Bus Schedule,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07 - Los Angeles,speedmap_segs_available,2024-10-16
2,Los Angeles County Metropolitan Transportation...,LA Metro Rail Schedule,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,07 - Los Angeles,speedmap_segs_available,2024-10-16


In [11]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [12]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
42,Los Angeles County Metropolitan Transportation...,182,recPnGkwdpnr8jmHB,07 - Los Angeles,True,2024-10-16,0666caf3ec1ecc96b74f4477ee4bc939,speedmap_segs_available


In [13]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'
speedmap_segs = gpd.read_parquet(path) #  aggregated
# speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [17]:
shared_utils.schedule_rt_utils.filter_dim_county_geography(analysis_date)

,organization_name,caltrans_district
0,Inyo County,09 - Bishop
1,City of Bishop,09 - Bishop
2,Bishop Paiute Tribe,09 - Bishop
3,Timbisha Shoshone Tribe,09 - Bishop
4,Lone Pine Paiute-Shoshone Tribe,09 - Bishop
...,...,...
1284,Cambria Community Council,05 - San Luis Obispo
1285,San Luis Obispo Council of Governments,05 - San Luis Obispo
1286,San Luis Obispo Regional Transit Authority,05 - San Luis Obispo
1287,United Cerebral Palsy of San Luis Obispo County,05 - San Luis Obispo


In [18]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'direction_id', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'n_trips_sch',
       'trips_hr_sch', 'route_short_name'],
      dtype='object')

In [18]:
#  TODO to utils
def time_period_and_arrowize(gdf, time_period):
    
    gdf = gdf >> filter(_.time_of_day == time_period)
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [19]:
time_of_day = 'AM Peak'

In [50]:
period_test = time_period_and_arrowize(speedmap_segs, time_of_day)

In [51]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [52]:
period_test['fast_slow_ratio'] = period_test.p80_mph / period_test.p20_mph

In [68]:
period_test.fast_slow_ratio = period_test.fast_slow_ratio.replace(np.inf, 3)

In [69]:
period_test = period_test.round(1)

In [70]:
period_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name,fast_slow_ratio
1080,015d67d5b75b5cf2b710bbadadfb75f5,9ce7f6d9232b32ee1401935c947a1e11,3,17,0.0,40097__40095,Bridgeway & Napa St__Bridgeway & Easterby St,40097-40095-1,AM Peak,13.1,...,Bay Area 511 Marin Schedule,04 - Oakland,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-218717.778 -14345.020, -218718.216 ...",1,0.3,17,1.0
1081,015d67d5b75b5cf2b710bbadadfb75f5,9ce7f6d9232b32ee1401935c947a1e11,3,17,0.0,40095__40092,Bridgeway & Easterby St__Bridgeway & Nevada St,40095-40092-1,AM Peak,22.8,...,Bay Area 511 Marin Schedule,04 - Oakland,recNOb7pqBRlQVG5e,Marin County Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-219100.744 -14266.315, -219102.537 ...",1,0.3,17,1.0


## export map

In [71]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [72]:
#  TODO clone render method from RtFilterMapper, to utils
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/194_am_peak_new.geojson.gz


In [73]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs AM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/194_am_peak_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.98273520932856, -122.54536343118872),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIEFNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xOTRfYW1fcGVha19uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzcuOTgyNzM1MjA5MzI4NTYsIC0xMjIuNTQ

In [79]:
#  TODO clone render method from RtFilterMapper, to utils
new_speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='new_speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/194_am_peak_new.geojson.gz


In [75]:
new_speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs AM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/194_am_peak_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'new_speedmap'}],
  'lat_lon': (37.98273520932856, -122.54536343118872),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIEFNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xOTRfYW1fcGVha19uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJuZXdfc3BlZWRtYXAifV0sICJsYXRfbG9uIjogWzM3Ljk4MjczNTIwOTMyODU2LCA

In [80]:
#  TODO clone render method from RtFilterMapper, to utils
new_speedmap_state_var = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new_var', map_type='new_speed_variation',
                                    color_col='fast_slow_ratio', cmap=shared_utils.rt_utils.VARIANCE_FIXED_COLORSCALE,
                                    legend_url=shared_utils.rt_utils.VARIANCE_LEGEND_URL,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/194_am_peak_new_var.geojson.gz


In [81]:
new_speedmap_state_var

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs AM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/194_am_peak_new_var.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5},
    'type': 'new_speed_variation'}],
  'lat_lon': (37.98273520932856, -122.54536343118872),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/variance_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIEFNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xOTRfYW1fcGVha19uZXdfdmFyLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX0sICJ0eXBlIjogIm5ld19zcGVlZF92YXJpYXRpb24ifV0sICJsYXRfbG9uIjogWzM3Ljk4MjczNTIwOTMyODU2LCAtMTIyLjU0NTM2MzQzMTE4ODcyXSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiOiAiaHR0c

In [78]:
period_test.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'direction_id', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'n_trips_sch',
       'trips_hr_sch', 'route_short_name', 'fast_slow_ratio'],
      dtype='object')

## docs!

In [28]:
domain = cmap.index

In [29]:
range_ = [cmap.rgb_hex_str(i) for i in cmap.index]

In [30]:
df = speedmap_segs[['time_of_day', 'p50_mph', 'p20_mph', 'p80_mph']]
df = df >> group_by(_.time_of_day) >> summarize(p50_mph = _.p50_mph.quantile(.5),
                                               p20_mph = _.p20_mph.quantile(.5),
                                               p80_mph = _.p80_mph.quantile(.5),)
df['p50 - p20'] = -(df['p50_mph'] - df['p20_mph'])
df['p80 - p50'] = df['p80_mph'] - df['p50_mph']

In [31]:
error_bars = alt.Chart(df).mark_errorbar(thickness=5, color='gray', opacity=.6).encode(
    y = alt.Y("p50_mph:Q", title='Segment Speed (mph): 20, 50, 80%ile'),
    yError=("p50 - p20:Q"),
    yError2=("p80 - p50:Q"),
    x = alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
    tooltip=[alt.Tooltip('p20_mph:Q', title="p20 mph"), alt.Tooltip('p50_mph:Q', title="p50 mph"),
            alt.Tooltip('p80_mph:Q', title="p80 mph")]
).properties(width=400)

points = alt.Chart(df).mark_point(filled=True, size = 300, opacity = 1).encode(
    alt.Y("p50_mph:Q"),
    alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl'],
         title='Time of Day'),
    color=alt.Color('p50_mph', title='Median Segment Speed (mph)').scale(domain=domain, range = range_),
    tooltip=[alt.Tooltip('p50_mph:Q', title="p50 mph")],
)

chart = error_bars + points
chart = chart.configure(axis = alt.AxisConfig(labelFontSize=14, titleFontSize=18),
                       legend = alt.LegendConfig(titleFontSize=14, labelFontSize=14, titleLimit=250,
                                                 titleOrient='left', labelOffset=100))

In [32]:
chart

alt.LayerChart(...)